In [90]:
import numpy as np 
import pandas as pd 
import requests
import xlsxwriter
import math
pd.set_option('display.max_columns', None)

In [91]:
tickers = pd.read_csv("/Users/vladimirbarshchuk/Downloads/tickers - Sheet1.csv")

#Data Cleansing
fresh=tickers.to_numpy()
tickers.loc[-1]=["Ticker"]
tickers.index = tickers.index +1
tickers.sort_index(inplace=True)
new_header = tickers.loc[0]
tickers = tickers[0:]
tickers.columns = new_header
tickers.loc[0] = "AAPL"
tickers.columns = ["Ticker"]
tickers = tickers.sort_values(by="Ticker", ascending=True)

In [92]:
#setting up the requirements
symbol = "ZTS"
Token = "Tpk_059b97af715d417d9f49f50b51b1c448"
base_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={Token}"
data = requests.get(base_url).json()

In [93]:
#extractions from API request
price = data["latestPrice"]
market_cap = data['marketCap']

In [94]:
my_columns = ["Ticker", "Stock Price", "Market Capitalization", "Number of Shares to Buy"]
df = pd.DataFrame(columns=my_columns)
df.append(pd.Series([
    symbol,
    price,
    market_cap,
    "N/A"
],index=my_columns), ignore_index=True)

/var/folders/_b/qkbr_1g922s5wvd3yzfwj1b80000gn/T/ipykernel_21803/4270921776.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(pd.Series([


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,ZTS,153.05,69695512636,N/A


In [44]:
df_new = pd.DataFrame(columns = my_columns)

for symbol in tickers["Ticker"]:
    base_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={Token}"
    data = requests.get(base_url).json()
    df_new = df_new.append(
        pd.Series(
        [
            symbol,
            data['latestPrice'],
            data["marketCap"],
            'N/A'
        ],
        index = my_columns),
    ignore_index = True
    )

/var/folders/_b/qkbr_1g922s5wvd3yzfwj1b80000gn/T/ipykernel_21803/1634345239.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new = df_new.append(
/var/folders/_b/qkbr_1g922s5wvd3yzfwj1b80000gn/T/ipykernel_21803/1634345239.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new = df_new.append(
/var/folders/_b/qkbr_1g922s5wvd3yzfwj1b80000gn/T/ipykernel_21803/1634345239.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new = df_new.append(
/var/folders/_b/qkbr_1g922s5wvd3yzfwj1b80000gn/T/ipykernel_21803/1634345239.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new = df_new.append(
/var/folders/_b/qkbr_1g922s5wvd3yzfw

In [95]:
df_new

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,132.78,39954289917,N/A
1,AAL,13.95,9199936242,N/A
2,AAP,178.90,10306288977,N/A
3,AAPL,147.78,2403524971521,N/A
4,ABBV,150.21,263998718734,N/A
...,...,...,...,...
498,YUM,113.52,32958623223,N/A
499,ZBH,110.08,23532355934,N/A
500,ZBRA,273.15,13915650964,N/A
501,ZION,48.58,7370268327,N/A


In [96]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i +n]


In [97]:
#Using Batch API 

symbol_groups =list(chunks(tickers["Ticker"],100))
symbol_strings =[]
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

df_new=pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={Token}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        df_new = df_new.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
df_new

/var/folders/_b/qkbr_1g922s5wvd3yzfwj1b80000gn/T/ipykernel_21803/2549675910.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new = df_new.append(
/var/folders/_b/qkbr_1g922s5wvd3yzfwj1b80000gn/T/ipykernel_21803/2549675910.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new = df_new.append(
/var/folders/_b/qkbr_1g922s5wvd3yzfwj1b80000gn/T/ipykernel_21803/2549675910.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new = df_new.append(
/var/folders/_b/qkbr_1g922s5wvd3yzfwj1b80000gn/T/ipykernel_21803/2549675910.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new = df_new.append(
/var/folders/_b/qkbr_1g922s5wvd3

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,132.29,39031215699,N/A
1,AAL,14.07,9351259921,N/A
2,AAP,174.44,10337359139,N/A
3,AAPL,151.70,2461766023649,N/A
4,ABBV,153.03,269492277049,N/A
...,...,...,...,...
498,YUM,114.97,33003946140,N/A
499,ZBH,109.95,23529737038,N/A
500,ZBRA,269.99,13773614821,N/A
501,ZION,48.88,7315205206,N/A


In [98]:
portfolio_size = input("Enter the value of your portfolio")

try:
    val = float(portfolio_size)
    print(val)
except ValueError:
    print("Please Enter a Numerical Value \nPlease try again:")
    portfolio_size = input("Enter the value of your portfolio")
    val = float(portfolio_size)

Enter the value of your portfolio10000000
10000000.0


In [137]:
position_size = val/len(df_new.index)

for i in range(0, len(df_new.index)):
    df_new.loc[i, "Number of Shares to Buy"] = math.floor(position_size/df_new.loc[i, "Stock Price"])
df_new

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,132.29,39031215699,150
1,AAL,14.07,9351259921,1412
2,AAP,174.44,10337359139,113
3,AAPL,151.70,2461766023649,131
4,ABBV,153.03,269492277049,129
...,...,...,...,...
498,YUM,114.97,33003946140,172
499,ZBH,109.95,23529737038,180
500,ZBRA,269.99,13773614821,73
501,ZION,48.88,7315205206,406


In [138]:
out_path = r"/Users/vladimirbarshchuk/Desktop/Python_Algo_Trading/Recommended trade.xlsx"

writer = pd.ExcelWriter(out_path, engine = "xlsxwriter")

df_new.to_excel(writer, "Recommended Trades",  index =False)

In [139]:
background_color = "#178BAE"
font_color = "#ffffff"

workbook  = writer.book

string_format = workbook.add_format(
    {
        "font_color" : font_color,
        "bg_color": background_color,
        "border" : 1
    }
)

dollar_format = workbook.add_format(
    {
        "num_format":"$0.00",
        "font_color": font_color,
        "bg_color": background_color,
        "border":1
    }
)

integer_format = workbook.add_format(
    {
        "num_format":"0",
        "font_color": font_color,
        "bg_color": background_color,
        "border":1
    }
)



In [140]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)


In [141]:
writer.save()